### Import library

In [1]:
import os
os.chdir('../../')
os.getcwd()

'C:\\Users\\csia7\\OneDrive\\문서\\GitHub\\WQBrain_2024_API'

In [2]:
import ace_lib as ace
import helpful_functions as hf
import pandas as pd
import requests
import plotly.express as px
import pygwalker as pyg
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

### Start session
Enter credentials once - they will be saved to local folder and loaded each time

In [3]:
s = ace.start_session()

Complete biometrics authentication and press any key to continue: 
https://api.worldquantbrain.com/authentication/persona?inquiry=inq_8sprG6tDj7L4UcduisqWJk5k5gBJ

 


## Global region Alpha Template

In [7]:
expression_template = f'''
group = (country+1)*group_max(pv13_52_minvol_1m_all_delay_1_sector, market) + pv13_52_minvol_1m_all_delay_1_sector;
group_neutralize(ts_scale(group_backfill(fnd23_intfvalld1_ecns, industry, 252, std=1), 252), densify(group))
'''

#### Step 1. Download datsets

In [4]:
datasets_df = hf.get_datasets(s, region = 'GLB', universe = 'MINVOL1M') # by default we load all datasets USA TOP3000 delay 1    , region='GLB', universe='MINVOL1M'
datasets_df # DataFrame.head() shows first 5 rows of the dataframe 

,id,name,description,category,subcategory,region,delay,universe,coverage,turnover,valueScore,userCount,alphaCount,fieldCount,themes,researchPapers
0,analyst11,ESG scores,Environmental Social Governance scores that ex...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-esg', 'name': 'ESG'}",GLB,1,MINVOL1M,0.7917,None,4.0,66,299,197,[],"[{'title': 'Research Paper 19: ESG Preference,..."
1,analyst14,Estimations of Key Fundamentals,This dataset reports many items from financial...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-analyst-estimates', 'name': 'A...",GLB,1,MINVOL1M,0.5186,None,3.0,101,584,926,[],[{'title': 'Research Paper 10: Investor Learni...
2,analyst15,Earnings forecasts,This dataset provides bottom-up forecast data ...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-analyst-estimates', 'name': 'A...",GLB,1,MINVOL1M,0.9929,None,2.0,157,1489,288,[],[]
3,analyst16,Real Time Estimates,This dataset provides real-time access to the ...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-crowdsourced-estimates', 'name...",GLB,1,MINVOL1M,0.9013,None,2.0,134,807,42,[],[]
4,analyst35,ESG Model,The dataset provide ESG related information ba...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-esg', 'name': 'ESG'}",GLB,1,MINVOL1M,0.4375,None,3.0,63,201,23,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,shortinterest6,SmartHoldings Model,This dataset is a global stock selection model...,"{'id': 'shortinterest', 'name': 'Short Interest'}","{'id': 'shortinterest-short-sale-models', 'nam...",GLB,1,MINVOL1M,0.9711,None,2.0,91,382,11,[],[]
84,shortinterest7,Short Selling Model,The dataset combines features obtained from sh...,"{'id': 'shortinterest', 'name': 'Short Interest'}","{'id': 'shortinterest-short-sale-models', 'nam...",GLB,1,MINVOL1M,1.0000,None,2.0,73,331,16,[],[{'title': 'Research Paper 04: Strategic Rebal...
85,socialmedia12,Sentiment Data for Equity,This dataset provides sentiment data with diff...,"{'id': 'socialmedia', 'name': 'Social Media'}","{'id': 'socialmedia-social-media', 'name': 'So...",GLB,1,MINVOL1M,0.6731,None,3.0,17,25,2,[],[{'title': 'Research Paper 01: Textual Sentime...
86,socialmedia5,Lexical Breakdown Data,Sentiment scores derived from social media. So...,"{'id': 'socialmedia', 'name': 'Social Media'}","{'id': 'socialmedia-social-media', 'name': 'So...",GLB,1,MINVOL1M,0.3753,None,5.0,2,3,12,[],[]


In [5]:
# select needed datasets
selected_datasets_df_1 = datasets_df[
    (datasets_df["name"].str.contains('relationship data for equity', case=False))
].sort_values(by=['valueScore'], ascending=False)
selected_datasets_df_1

,id,name,description,category,subcategory,region,delay,universe,coverage,turnover,valueScore,userCount,alphaCount,fieldCount,themes,researchPapers
73,pv13,Relationship Data for Equity,The dataset outputs various classifications an...,"{'id': 'pv', 'name': 'Price Volume'}","{'id': 'pv-relationship', 'name': 'Relationship'}",GLB,1,MINVOL1M,0.9152,None,1.0,127,1924,29,[],[]


In [6]:
# select needed datasets
selected_datasets_df_2 = datasets_df[
    (datasets_df["name"].str.contains('fundamental point in time', case=False))
].sort_values(by=['valueScore'], ascending=False)
selected_datasets_df_2

,id,name,description,category,subcategory,region,delay,universe,coverage,turnover,valueScore,userCount,alphaCount,fieldCount,themes,researchPapers
21,fundamental23,Fundamental Point in Time Data,This dataset gives point in time access to fun...,"{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,0.6704,None,2.0,257,2041,886,[],[{'title': 'Research Paper 01: Textual Sentime...


#### Step 2. Select the needed datafields

##### Step 2-1. equity relationship datafield extraction

In [7]:
dataset_id_1 = selected_datasets_df_1.id.values.tolist()[0] # create a list of selected datasets ids, our list has only one element
dataset_id_1

'pv13'

In [8]:
datafields_df_1 = hf.get_datafields(s, region = 'GLB', universe = 'MINVOL1M', dataset_id=dataset_id_1) # download all fields of dataset news
datafields_df_1

,id,description,dataset,category,subcategory,region,delay,universe,type,coverage,turnover,userCount,alphaCount,themes
0,pv13_10_f2_g3_minvol_1m_sector,grouping fields,"{'id': 'pv13', 'name': 'Relationship Data for ...","{'id': 'pv', 'name': 'Price Volume'}","{'id': 'pv-relationship', 'name': 'Relationship'}",GLB,1,MINVOL1M,GROUP,1.0000,None,27,146,[]
1,pv13_10_f2_g4_minvol_1m_sector,grouping fields,"{'id': 'pv13', 'name': 'Relationship Data for ...","{'id': 'pv', 'name': 'Price Volume'}","{'id': 'pv-relationship', 'name': 'Relationship'}",GLB,1,MINVOL1M,GROUP,1.0000,None,16,102,[]
2,pv13_10_f3_g2_minvol_1m_sector,grouping fields,"{'id': 'pv13', 'name': 'Relationship Data for ...","{'id': 'pv', 'name': 'Price Volume'}","{'id': 'pv-relationship', 'name': 'Relationship'}",GLB,1,MINVOL1M,GROUP,1.0000,None,17,98,[]
3,pv13_10_minvol_1m_sector,grouping fields,"{'id': 'pv13', 'name': 'Relationship Data for ...","{'id': 'pv', 'name': 'Price Volume'}","{'id': 'pv-relationship', 'name': 'Relationship'}",GLB,1,MINVOL1M,GROUP,1.0000,None,36,120,[]
4,pv13_10_sector,grouping fields,"{'id': 'pv13', 'name': 'Relationship Data for ...","{'id': 'pv', 'name': 'Price Volume'}","{'id': 'pv-relationship', 'name': 'Relationship'}",GLB,1,MINVOL1M,GROUP,1.0000,None,4,10,[]
5,pv13_1l_scibr,grouping fields,"{'id': 'pv13', 'name': 'Relationship Data for ...","{'id': 'pv', 'name': 'Price Volume'}","{'id': 'pv-relationship', 'name': 'Relationship'}",GLB,1,MINVOL1M,GROUP,0.8770,None,31,120,[]
6,pv13_20_minvol_1m_sector,grouping fields,"{'id': 'pv13', 'name': 'Relationship Data for ...","{'id': 'pv', 'name': 'Price Volume'}","{'id': 'pv-relationship', 'name': 'Relationship'}",GLB,1,MINVOL1M,GROUP,1.0000,None,36,234,[]
7,pv13_2_f3_g2_minvol_1m_sector,grouping fields,"{'id': 'pv13', 'name': 'Relationship Data for ...","{'id': 'pv', 'name': 'Price Volume'}","{'id': 'pv-relationship', 'name': 'Relationship'}",GLB,1,MINVOL1M,GROUP,1.0000,None,19,75,[]
8,pv13_2_f4_g3_minvol_1m_sector,grouping fields,"{'id': 'pv13', 'name': 'Relationship Data for ...","{'id': 'pv', 'name': 'Price Volume'}","{'id': 'pv-relationship', 'name': 'Relationship'}",GLB,1,MINVOL1M,GROUP,1.0000,None,21,88,[]
9,pv13_2_minvol_1m_all_delay_1_sector,grouping fields,"{'id': 'pv13', 'name': 'Relationship Data for ...","{'id': 'pv', 'name': 'Price Volume'}","{'id': 'pv-relationship', 'name': 'Relationship'}",GLB,1,MINVOL1M,GROUP,1.0000,None,26,88,[]


In [9]:
selected_datafields_df_1 = datafields_df_1[
    (datafields_df_1['type'] == 'GROUP') & 
    (datafields_df_1['userCount'] > 5) &
    (datafields_df_1['id'].str.contains('pv13_52_minvol_1m_all_delay_1_sector', case = False))
].sort_values(by=['userCount'], ascending=False)
selected_datafields_df_1

,id,description,dataset,category,subcategory,region,delay,universe,type,coverage,turnover,userCount,alphaCount,themes
15,pv13_52_minvol_1m_all_delay_1_sector,grouping fields,"{'id': 'pv13', 'name': 'Relationship Data for ...","{'id': 'pv', 'name': 'Price Volume'}","{'id': 'pv-relationship', 'name': 'Relationship'}",GLB,1,MINVOL1M,GROUP,1.0,None,22,125,[]


In [10]:
len(selected_datafields_df_1)

1

In [11]:
selected_datafields_df_1_id = selected_datafields_df_1.id.values.tolist()
selected_datafields_df_1_id

['pv13_52_minvol_1m_all_delay_1_sector']

##### Step 2-2. fundamental time datafields extraction

In [12]:
dataset_id_2 = selected_datasets_df_2.id.values.tolist()[0] # create a list of selected datasets ids, our list has only one element
dataset_id_2

'fundamental23'

In [13]:
datafields_df_2 = hf.get_datafields(s, region = 'GLB', universe = 'MINVOL1M', dataset_id=dataset_id_2) # download all fields of dataset news
datafields_df_2

,id,description,dataset,category,subcategory,region,delay,universe,type,coverage,turnover,userCount,alphaCount,themes
0,fnd23_acc_payable,accounts payable,"{'id': 'fundamental23', 'name': 'Fundamental P...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,MATRIX,0.8848,None,11,30,[]
1,fnd23_annfv_item,integer index of the item description field. Y...,"{'id': 'fundamental23', 'name': 'Fundamental P...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,VECTOR,0.4969,None,1,1,[]
2,fnd23_annfv_value,annual financial values.,"{'id': 'fundamental23', 'name': 'Fundamental P...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,VECTOR,0.4969,None,2,2,[]
3,fnd23_annfvmfm2_acta,total current assets. Sum of cash and short te...,"{'id': 'fundamental23', 'name': 'Fundamental P...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,MATRIX,0.9223,None,16,30,[]
4,fnd23_annfvmfm2_amao,Amortization of Acquisition Costs,"{'id': 'fundamental23', 'name': 'Fundamental P...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,MATRIX,0.3231,None,1,2,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881,fnd23_topic3,third tier topic mapped to parent topic.,"{'id': 'fundamental23', 'name': 'Fundamental P...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,VECTOR,0.7647,None,0,0,[]
882,fnd23_tot_assets,total assets.,"{'id': 'fundamental23', 'name': 'Fundamental P...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,MATRIX,1.0000,None,31,84,[]
883,fnd23_tot_inventory,total inventory.,"{'id': 'fundamental23', 'name': 'Fundamental P...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,MATRIX,0.9832,None,6,8,[]
884,fnd23_tot_revenue,total revenue.,"{'id': 'fundamental23', 'name': 'Fundamental P...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,MATRIX,0.9928,None,12,18,[]


In [14]:
selected_datafields_df_2 = datafields_df_2[
    (datafields_df_2['type'] == 'MATRIX') &
    (datafields_df_2["userCount"] > 5)
].sort_values(by=['userCount'], ascending=False)
selected_datafields_df_2

,id,description,dataset,category,subcategory,region,delay,universe,type,coverage,turnover,userCount,alphaCount,themes
14,fnd23_annfvmfm2_fcos,Changes in Working Capital,"{'id': 'fundamental23', 'name': 'Fundamental P...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,MATRIX,1.0000,None,70,167,[]
23,fnd23_annfvmfm2_olto,includes diverse cash flows that are reported ...,"{'id': 'fundamental23', 'name': 'Fundamental P...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,MATRIX,1.0000,None,41,101,[]
882,fnd23_tot_assets,total assets.,"{'id': 'fundamental23', 'name': 'Fundamental P...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,MATRIX,1.0000,None,31,84,[]
875,fnd23_roe,return on equity,"{'id': 'fundamental23', 'name': 'Fundamental P...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,MATRIX,1.0000,None,29,47,[]
870,fnd23_net_income,net income,"{'id': 'fundamental23', 'name': 'Fundamental P...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,MATRIX,1.0000,None,28,74,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,fnd23_annfvmfm2_lctl,represents current liabilities for industrial ...,"{'id': 'fundamental23', 'name': 'Fundamental P...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,MATRIX,0.9225,None,6,8,[]
302,fnd23_intfvalld1_ipos,Operating Income.,"{'id': 'fundamental23', 'name': 'Fundamental P...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,MATRIX,0.9922,None,6,13,[]
883,fnd23_tot_inventory,total inventory.,"{'id': 'fundamental23', 'name': 'Fundamental P...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,MATRIX,0.9832,None,6,8,[]
132,fnd23_intfvalld1_cdav,Deferred Revenue - Current,"{'id': 'fundamental23', 'name': 'Fundamental P...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,MATRIX,0.3804,None,6,15,[]


In [15]:
selected_datafields_df_2_id = selected_datafields_df_2.id.values.tolist()
selected_datafields_df_2_id

['fnd23_annfvmfm2_fcos',
 'fnd23_annfvmfm2_olto',
 'fnd23_tot_assets',
 'fnd23_roe',
 'fnd23_net_income',
 'fnd23_intfvalld1_iscs',
 'fnd23_cash_n_equivalents',
 'fnd23_aor',
 'fnd23_ebitda',
 'fnd23_intfvmfm2_olto',
 'fnd23_net_debt',
 'fnd23_ofc',
 'fnd23_intfvalld1_dtns',
 'fnd23_annfvmfm2_iscs',
 'fnd23_intfvmfm_olto',
 'fnd23_ve',
 'fnd23_capex',
 'fnd23_curr_assets',
 'fnd23_annfvmfm2_acta',
 'fnd23_annfvmfm2_cnin',
 'fnd23_intfvalld1_ecns',
 'fnd23_intfvmalld1_olto',
 'fnd23_ebit',
 'fnd23_annfvmfm2_bpao',
 'fnd23_intfvmalld1_sedv',
 'fnd23_annfvmfm_olto',
 'fnd23_tot_revenue',
 'fnd23_annfvmfm2_bloo',
 'fnd23_acc_payable',
 'fnd23_curr_liabilities',
 'fnd23_ffcf',
 'fnd23_intfvalld1_fbds',
 'fnd23_intfvalld1_ccns',
 'fnd23_annfvmfm2_ltio',
 'fnd23_debt_issuance',
 'fnd23_intfvmfm2_fcos',
 'fnd23_intfvmalld1_xbin',
 'fnd23_intfvmalld1_ipos',
 'fnd23_intfvalld1_fcos',
 'fnd23_annfvmfm2_eltq',
 'fnd23_annfvmfm2_inds',
 'fnd23_cogs',
 'fnd23_lt_debt',
 'fnd23_intfvmalld1_ccns',
 'f

#### Step 4. Apply generate_alpha function to the expression list
In generate alpha function you can specify region, universe, decay, delay and other simulation settings

이제 만들어둔 expressions 을 사용해 alpha simulation 을 해줍니다.

In [61]:
#?ace.generate_alpha

In [32]:
equity, time = ' ', ' '

expression_template = f'''
group = (country+1)*group_max({equity}, market) + {equity};
group_neutralize(ts_scale(group_backfill({time}, industry, 252, std=1), 252), densify(group))
'''

In [16]:
expressions = []
equity = selected_datafields_df_1_id[0]
for time in selected_datafields_df_2_id:
    expressions.append(f'group = (country+1)*group_max({equity}, market) + {equity};group_neutralize(ts_scale(group_backfill({time}, industry, 252, std=1), 252), densify(group))')

In [20]:
len(expressions)

65

In [18]:
#when you send multiple alphas for simulation, please make sure all alphas of a single list should have common settings
#alphas with different settings should be sent in a different list, for instance below list has all alphas with same settings

alpha_list = [ace.generate_alpha(x, region= "GLB", universe = "MINVOL1M", neutralization = 'COUNTRY', truncation = 0.01, delay = 1, decay = 0) for x in expressions]
alpha_list[0]

{'type': 'REGULAR',
 'settings': {'instrumentType': 'EQUITY',
  'region': 'GLB',
  'universe': 'MINVOL1M',
  'delay': 1,
  'decay': 0,
  'neutralization': 'COUNTRY',
  'truncation': 0.01,
  'pasteurization': 'ON',
  'testPeriod': 'P0Y0M0D',
  'unitHandling': 'VERIFY',
  'nanHandling': 'OFF',
  'language': 'FASTEXPR',
  'visualization': False},
 'regular': 'group = (country+1)*group_max(pv13_52_minvol_1m_all_delay_1_sector, market) + pv13_52_minvol_1m_all_delay_1_sector;group_neutralize(ts_scale(group_backfill(fnd23_annfvmfm2_fcos, industry, 252, std=1), 252), densify(group))'}

### Simulate alpha list, get simulation result

simulate_alpha_list_multi will do a multi-simulation if list of alphas is greater than 10, which is the case here

the returned object will contain simulation results for all alphas as a list

In [21]:
#alpha expressions are sliced to first 10 for demonstration purpose

result = ace.simulate_alpha_list_multi(s, alpha_list)

100%|███████████████████████████████████████████████████████████████████████████████| 22/22 [1:23:55<00:00, 228.89s/it]


위 코드를 실행하면 시뮬레이션이 시작됩니다. 100개의 알파에 대략 1시간 정도 소요되니, 인터넷 연결이 끊기지 않게 주의하시고, 너무 많은 알파를 한 번에 돌리기보다는 몇개씩 끊어서 돌려도 좋습니다.

In [22]:
#prettify_result function can be used from the helpful_functions library to take a look at IS stats of all the simulated alphas

result_st1 = hf.prettify_result(result, detailed_tests_view=False)
result_st1

,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,start_date,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover
0,5312939,20000000,4041,4022,0.0319,0.0513,0.0751,0.003220,1.26,1.96,2012-01-22,AwopJ9d,group = (country+1)*group_max(pv13_52_minvol_1...,PASS,PASS,FAIL,PASS,PASS,PASS,PASS
1,5281117,20000000,4062,3999,0.0313,0.0510,0.0704,0.003263,1.26,1.98,2012-01-22,xxJLwqb,group = (country+1)*group_max(pv13_52_minvol_1...,PASS,PASS,FAIL,PASS,PASS,PASS,PASS
2,5508931,20000000,4056,4001,0.0300,0.0532,0.0827,0.003548,1.22,1.87,2012-01-22,273gaOP,group = (country+1)*group_max(pv13_52_minvol_1...,PASS,PASS,FAIL,PASS,PASS,PASS,PASS
3,3622137,20000000,4074,3938,0.0266,0.0350,0.0260,0.002626,1.21,2.29,2012-01-22,EwjlvM9,\ngroup = (country+1)*group_max(pv13_52_minvol...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
4,3622137,20000000,4074,3938,0.0266,0.0350,0.0260,0.002626,1.21,2.29,2012-01-22,5wPmal1,group = (country+1)*group_max(pv13_52_minvol_1...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,-844398,20000000,4238,3688,0.0400,-0.0082,0.1156,-0.000408,-0.11,-0.43,2012-01-22,8zMxdvv,group = (country+1)*group_max(pv13_52_minvol_1...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS
61,-905431,20000000,3718,3808,0.0259,-0.0087,0.1512,-0.000676,-0.15,-0.58,2012-01-22,VpAb6kM,group = (country+1)*group_max(pv13_52_minvol_1...,PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS
62,-899515,20000000,3578,3661,0.0250,-0.0087,0.1213,-0.000696,-0.16,-0.60,2012-01-22,5wPd2M6,group = (country+1)*group_max(pv13_52_minvol_1...,PASS,PASS,FAIL,FAIL,FAIL,PASS,PASS
63,-3259252,20000000,4119,3936,0.0248,-0.0315,0.3724,-0.002537,-0.69,-1.38,2012-01-22,MpJNW6L,group = (country+1)*group_max(pv13_52_minvol_1...,PASS,PASS,FAIL,FAIL,FAIL,FAIL,PASS


prettify 함수를 이용해 dataframe 형식으로 변환하면 결과를 보기 더 편합니다.

In [23]:
result_st1.to_csv('2024_09_07_GLB.csv')

### How to submit?

Create a list of submittable alphas - alphas that have no FAIL in is_tests

시뮬레이션을 마친 알파 중 제출 가능한 알파들을 제출해 봅시다.

In [24]:
#to take a look at the combined result of all new alphas

is_tests_df = hf.concat_is_tests(result)
is_tests_df

,alpha_id,endDate,limit,message,name,result,startDate,themes,value,year
0,RpGox8g,NaN,1.58,NaN,LOW_SHARPE,PASS,NaN,NaN,1.6800,NaN
1,RpGox8g,NaN,1.00,NaN,LOW_FITNESS,FAIL,NaN,NaN,0.8000,NaN
2,RpGox8g,NaN,0.01,NaN,LOW_TURNOVER,PASS,NaN,NaN,0.0307,NaN
3,RpGox8g,NaN,0.70,NaN,HIGH_TURNOVER,PASS,NaN,NaN,0.0307,NaN
4,RpGox8g,NaN,NaN,NaN,CONCENTRATED_WEIGHT,PASS,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
840,Ew9jG3r,NaN,NaN,NaN,DATA_DIVERSITY,PENDING,NaN,NaN,NaN,NaN
841,Ew9jG3r,NaN,NaN,NaN,PROD_CORRELATION,PENDING,NaN,NaN,NaN,NaN
842,Ew9jG3r,NaN,NaN,NaN,REGULAR_SUBMISSION,PENDING,NaN,NaN,NaN,NaN
843,Ew9jG3r,2020-01-25,1.58,NaN,IS_LADDER_SHARPE,FAIL,2022-01-24,NaN,1.5800,2.0


In [25]:
#making a list of failed alphas
failed_alphas = is_tests_df.query('result=="FAIL"')['alpha_id'].unique()

#making a list of passed alphas
passed_alphas = list(set(is_tests_df['alpha_id']).difference(failed_alphas))

print(f'Failed alphas:{failed_alphas}\nPassed alphas:{passed_alphas}')

Failed alphas:['RpGox8g' 'WpvRrVd' '273xVN5' 'ad58ma1' 'xxJo7vW' 'dxN6J92' 'Lp0wodv'
 'WpvRrAd' 'g1nVEmK' 'jZqKbno' '9wNOLJ1' 'MpJNW6L' 'dxNrX92' 'OplXY6b'
 'jZqKbeE' '8zMxdvv' 'jZqKbYE' 'Awo8Jzl' 'g1nVqGl' 'XpdAYw8' 'VpAbYqJ'
 'qAQJmvO' '7weRK9O' '8zMxK1v' 'Ew9PrAr' 'Awo8wJY' 'Ppkmprx' '7weRw0O'
 'KbWMO8k' 'jZqK3wO' '378v9n6' 'ad59Lk2' 'VpAb6kM' 'k3z6VEd' 'Jp6J3al'
 'qAQJ52j' 'Ew9Pmn9' 'NpYl2Ap' 'm8EWlZ1' 'Lp0Yrxe' 'g1nVdzM' 'XpdAeEz'
 'roZMnA3' '5wPd2M6' 'RpGOnO1' 'RpGOnOz' '071V0Jv' 'xxJLwOm' 'AwopJ9d'
 'xxJLwqb' '273gaOP' 'dxNMEOx' 'lVon7m7' 'AwopnAe' '8zM0OMo' 'p6926l3'
 'vLWELv3' 'Qawxa95' 'PpkRGEx' 'Ew9jG3r']
Passed alphas:['EwjlvM9', '273o2J5', '378v2AZ', '5wPmal1', 'lVoYwWl']


In [5]:
for alpha_id in ['ad59v2O', 'k3z6qqk', 'Lp0YPva', '9wNOzNr', '7weR901', '6w5Ln6E', 'QawpLe5', '17rAQ6m', '273obEY', 'xxJpKLg', 'Wpv2EZo', 'm8EWMk1']:
    hf.set_alpha_properties(s, alpha_id, tags = ['pass_Sep07_mixing'])

In [27]:
for alpha_id in passed_alphas:
    hf.set_alpha_properties(s, alpha_id, tags = ['Sep07_GLB'])

위 함수로 통과한 알파들에 태그를 붙일 수 있습니다.

When you got a list of submittable alphas, you can call function submit_alpha()

In [50]:
#calling submit_alpha on all alphas that have passed the submission tests

submit_result = {alpha_id: ace.submit_alpha(s, alpha_id) for alpha_id in passed_alphas}

위 함수로 알파를 제출할 수 있습니다.

알파의 id 로 제출하는 것이기 때문에, 그 알파를 시뮬레이션 한 날짜가 중요합니다. 가령, 1주일 전에 시뮬레이션 해 결과를 저장해 두었던 alpha id로 제출을 한다면 제출 날짜가 1주일 전이 됩니다. 그러니 오늘 제출하고 싶은 alpha 가 있다면 시뮬레이션을 다시 돌려 새로 생긴 alpha id 를 이용해야 합니다. 이 과정은 alpha 에 특정 태그를 달아 Brain 플랫폼에서 직접 하는게 훨씬 수월합니다. (다만, 플랫폼에서는 alpha id 로 알파를 검색할 수 없습니다.)

In [51]:
#submit_result will have return values from the submit_result function

submit_result

{'ZpO36rQ': False}

주의하셔야 하실 점은, 하루 최대 알파 제출 한도인 4개를 루프중에 이미 넘기면 그 뒤의 알파들이 제출 가능하더라도 자동으로 제출에 실패하게 됩니다.

### Library Fuctions.

following are some other functions that you can use for your own analysis

**get_alpha_pnl(s, alpha_id)** - to get the pnl for an alpha

**get_alpha_yearly_stats(s, alpha_id)** - to get yearly statistics for an alpha

**get_self_corr(s, alpha_id)** - to get self correlation results for an alpha

**get_prod_corr(s, alpha_id)** - to get prod correlation results for an alpha

**get_check_submission(s, alpha_id)** - to get check submission result for an alpha

**check_self_corr_test(s, alpha_id)** - to check if alpha passes self correlation test (self_corr<0.7)

**check_prod_corr_test(s, alpha_id)** - to check if alpha passes prod correlation test (prod_corr<0.7)

**perfomance_comparison(s, alpha_id)** - to get the result of performance comparison for an alpha merged performance

유용한 추가 함수들입니다. prod correlation 혹은 self correlation 만을 확인하고 싶을 때, 혹은 알파의 pnl 을 확인하고 싶을때 유용히 쓰입니다.